In [20]:
import pandas as pd
import numpy as np
from scipy.stats import trim_mean # Die Funktion, die wir für die Aggregation brauchen

# --- Daten von LE2 laden ---
# Lade die bereinigten Daten, die du gerade gespeichert hast.
try:
    energy_df = pd.read_parquet('energy_cleaned.parquet')
    weather_df = pd.read_parquet('weather_cleaned.parquet')
    print("Bereinigte Daten (LE2) erfolgreich geladen.")
    print(f"Energy DataFrame Größe: {energy_df.shape}")
except FileNotFoundError:
    print("FEHLER: 'energy_cleaned.parquet' oder 'weather_cleaned.parquet' nicht gefunden.")
    print("Stelle sicher, dass die Dateien im selben Ordner wie dieses Notebook liegen.")

Bereinigte Daten (LE2) erfolgreich geladen.
Energy DataFrame Größe: (35064, 26)


## 4.4. Korrigierte Strategie: Cross-Sectional Trimmed Mean
Die ursprüngliche Intraday Trimmed Mean Aggregation der Temperatur reichte nicht aus, um den Datensatz auf die Länge des Energie-DF zu kürzen. Wir benötigen eine einzige Temperaturbeobachtung pro Tag.

Daher führen wir eine zweistufige Aggregation durch, um eine repräsentative regionale Temperatur zu schätzen:

Stufe 1 (Intraday): Die stündlichen Temperaturen jeder Stadt werden zu einem täglichen Durchschnitt aggregiert.

Stufe 2 (Cross-Sectional): Für jeden Tag werden die fünf täglichen Stadtdurchschnitte herangezogen. Wir wenden eine spezifische Trimmed Mean an, indem wir die höchste und die niedrigste Temperatur (die Extremwerte) entfernen und den Mittelwert der verbleibenden drei Städte bilden.

Diese Methode ist eine robuste Schätzung der Populationstendenz, die Extremwerte in den äußeren Regionen ignoriert, und führt zu einer konsistenten Tagesfrequenz , die dem Energiedatensatz entspricht.

## 4.5. Implementierung der korrigierten Trimmed Mean
Wir definieren nun die spezielle Aggregationsfunktion und wenden sie in den zwei Stufen an.

In [21]:
# --- 1. Hilfsfunktion für die Trimmed Mean ---
# Beibehalten der T-Mean-Funktion für die Energielast (T-Mean 10% pro Seite)
def t_mean_10(data):
    if len(data) < 10:
        return np.nan
    return trim_mean(data, proportiontocut=0.10)

# --- 2. Definition der Spezialfunktion für die 5 Städte ---
def t_mean_3_cities(data):
    """Entfernt den kleinsten und größten Wert aus 5 Städten und berechnet den Mean der restlichen 3."""
    data = data.dropna()
    # Sicherheitscheck: Muss mindestens 3 Einträge für die Mittelwertbildung haben
    if len(data) < 3: 
        return np.nan 
        
    # Sortieren, um den kleinsten (Index 0) und größten (Index -1) Wert zu entfernen
    sorted_data = np.sort(data)
    # Mittelwert der verbleibenden Werte (von Index 1 bis zum vorletzten Index -1)
    return np.mean(sorted_data[1:-1])

print("Funktionen für Trimmed Mean definiert.")

Funktionen für Trimmed Mean definiert.


## 4.6. Korrigierte Strategie: Aggregation und Cross-Sectional Trimmed Mean
Wir setzen die Energiedaten lediglich auf die Tagesebene um und wenden die robuste Trimmed Mean nur auf die Temperaturdaten an (Cross-Sectional über die 5 Städte).

In [24]:
from scipy.stats import trim_mean

# --- 1. Hilfsfunktion für die Intraday Trimmed Mean (10% gesamt) ---
def t_mean_10_intraday(data):
    """Trimmed Mean 10% gesamt (5% pro Seite) für stündliche Daten."""
    data = data.dropna()
    if len(data) < 10:
        return np.nan
    return trim_mean(data, proportiontocut=0.05) # 0.05 = 5% pro Seite

# --- 2. Definition der Spezialfunktion für die 5 Städte (Cross-Sectional) ---
def t_mean_3_cities(data):
    """Entfernt den kleinsten und größten Wert aus 5 Städten und berechnet den Mean der restlichen 3."""
    data = data.dropna()
    if len(data) < 3: 
        return np.nan 
        
    sorted_data = np.sort(data)
    # Mittelwert der verbleibenden Werte (Index 1 bis zum vorletzten Index -1)
    return np.mean(sorted_data[1:-1])

print("Spezialfunktionen für die Aggregation definiert.")

Spezialfunktionen für die Aggregation definiert.


## 4.7. Durchführung der Aggregation

In [25]:
# --- 1. Aggregation der Energielast (Intraday T-Mean 10%) ---
# Umstellung von stündlich auf täglich mit T-Mean 10% (robuster als Mean)
final_energy_trimmed = (
    energy_df['total load actual']
    .resample('D')
    .apply(t_mean_10_intraday) # NEU: Anwendung der T-Mean
    .to_frame(name='TotalLoad_TMean')
)

print("\nEnergielast-Aggregation (Intraday T-Mean) abgeschlossen.")
print(f"Länge des Energiedatensatzes: {final_energy_trimmed.shape[0]}")


# --- 2. Cross-Sectional Trimmed Mean für Wetter (Zwei Stufen) ---

# Stufe 1: Intraday Aggregation (Hourly zu Daily T-Mean 10% pro Stadt)
# Macht die Tageswerte robust gegen stündliche Messausreißer.
daily_temp_per_city = (
    weather_df
    .groupby('city_name')['temp']
    .resample('D')
    .apply(t_mean_10_intraday) # NEU: Anwendung der T-Mean
    .to_frame(name='DailyTemp_TMean_PerCity')
)

# Stufe 2: Cross-Sectional Aggregation (Trimmed Mean der 5 Städte)
# Entfernt regionale Extreme (kälteste/wärmste Stadt).
daily_temp_unstacked = daily_temp_per_city.unstack(level='city_name')
daily_temp_unstacked.columns = daily_temp_unstacked.columns.droplevel(0) 

# Anwendung der Spezialfunktion t_mean_3_cities zeilenweise (axis=1)
final_weather_trimmed_cross_sectional = (
    daily_temp_unstacked
    .apply(t_mean_3_cities, axis=1)
    .to_frame(name='DailyTemp_TMean_Regional')
)

print("\nWetter-Aggregation (Cross-Sectional T-Mean) abgeschlossen.")
print(f"Länge des Wetter-DF: {final_weather_trimmed_cross_sectional.shape[0]}")
print(final_weather_trimmed_cross_sectional.head())


Energielast-Aggregation (Intraday T-Mean) abgeschlossen.
Länge des Energiedatensatzes: 1462

Wetter-Aggregation (Cross-Sectional T-Mean) abgeschlossen.
Länge des Wetter-DF: 1462
                           DailyTemp_TMean_Regional
dt_iso                                             
2014-12-31 00:00:00+00:00                       NaN
2015-01-01 00:00:00+00:00                277.901433
2015-01-02 00:00:00+00:00                278.989829
2015-01-03 00:00:00+00:00                279.639496
2015-01-04 00:00:00+00:00                279.644540


## Finale Zusammenfassung:  Trimmed Mean Aggregation 
Um die Voraussetzungen für die Verknüpfung (LE4) zu schaffen und die Robustheit gegenüber Ausreißern zu gewährleisten, wurde eine zweifache Trimmed Mean Strategie auf die stündlichen Rohdaten angewandt.

#### 1. Energielast (Total Load)
Die stündlichen Energielastdaten wurden auf die Tagesebene umgestellt, um intraday (innerhalb des Tages) auftretende Lastspitzen zu neutralisieren.

Methode: Es wurde die Trimmed Mean (5% pro Seite, 10% Gesamt) angewandt.

Grund: Dies liefert einen robusteren Tagesmittelwert als der einfache arithmetische Mittelwert, da es extreme Stundenlasten ignoriert und somit die zentrale Tendenz des Lastverbrauchs repräsentiert.

Ergebnis: Ein TotalLoad_TMean pro Tag.

#### 2. Wetterdaten (Regionale Temperatur)
Die Temperaturdaten erforderten eine zweistufige Aggregation, um sowohl stündliche Messfehler als auch regionale Extreme zu eliminieren.

Stufe A: Intraday-Aggregation (Stündlich $\rightarrow$ Täglich pro Stadt)
Methode: Für jede der fünf Städte wurde die Trimmed Mean (5% pro Seite, 10% Gesamt) auf die stündlichen Werte angewandt.
Grund: Neutralisierung von stündlichen Messausreißern, um einen robusten täglichen Temperaturwert pro Stadt zu erhalten.

Stufe B: Cross-Sectional Aggregation (Über die 5 Städte)
Methode: Auf die fünf robusten täglichen Stadtdurchschnitte wurde die spezifische Trimmed Mean angewandt (Entfernen des kältesten und wärmsten Wertes, Mittelwert der verbleibenden drei).
Grund: Diese Methode liefert einen regional repräsentativen Wert, der Extremwerte in den äußeren Regionen ignoriert.Ergebnis: Ein einziger DailyTemp_TMean_Regional Wert pro Tag.

Beide DataFrames (final_energy_trimmed und final_weather_trimmed_cross_sectional) liegen nun auf der exakten täglichen Frequenz vor. Die Implementierung stellt sicher, dass jede Messung in der finalen Tabelle eine zweifach robuste Aggregation durchlaufen hat, was die Grundlage für das Verknüpfen (LE4) bildet.

In [26]:
final_energy_trimmed.head()

,TotalLoad_TMean
time,
2014-12-31 00:00:00+00:00,NaN
2015-01-01 00:00:00+00:00,23896.863636
2015-01-02 00:00:00+00:00,27126.863636
2015-01-03 00:00:00+00:00,25060.318182
2015-01-04 00:00:00+00:00,27173.500000


In [27]:
final_weather_trimmed_cross_sectional.head()

,DailyTemp_TMean_Regional
dt_iso,
2014-12-31 00:00:00+00:00,NaN
2015-01-01 00:00:00+00:00,277.901433
2015-01-02 00:00:00+00:00,278.989829
2015-01-03 00:00:00+00:00,279.639496
2015-01-04 00:00:00+00:00,279.644540


In [28]:
import os

# --- 1. Ordnerpfad definieren und erstellen ---
output_folder = 'data'

# Überprüfen, ob der Ordner 'data' existiert, andernfalls erstellen
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    print(f"Ordner '{output_folder}' wurde erfolgreich erstellt.")
else:
    print(f"Ordner '{output_folder}' existiert bereits.")

# --- 2. Abspeichern der finalen DataFrames ---

# Dateipfade definieren
energy_path = os.path.join(output_folder, 'LE3_Energy_TMean_Aggregated.parquet')
weather_path = os.path.join(output_folder, 'LE3_Weather_TMean_Aggregated.parquet')

# Daten speichern
final_energy_trimmed.to_parquet(energy_path)
final_weather_trimmed_cross_sectional.to_parquet(weather_path)

print("\n--- Speichern abgeschlossen! ✅ ---")
print(f"Die Energy T-Mean Daten wurden gespeichert unter: {energy_path}")
print(f"Die Wetter T-Mean Daten wurden gespeichert unter: {weather_path}")

Ordner 'data' wurde erfolgreich erstellt.

--- Speichern abgeschlossen! ✅ ---
Die Energy T-Mean Daten wurden gespeichert unter: data/LE3_Energy_TMean_Aggregated.parquet
Die Wetter T-Mean Daten wurden gespeichert unter: data/LE3_Weather_TMean_Aggregated.parquet
